In [3]:
import pandas as pd
from datetime import date, datetime
from datetime import datetime as dt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from io import StringIO
from selenium.webdriver.support.ui import Select

# update list

isna noktalarına try except
tefas çekimine 5-6 günlük geriye dönük merge (tatiller için)

# Fon Fiyatları Çekimi

In [4]:
driver = webdriver.Chrome()

start_date = pd.to_datetime('today')
end_date = pd.to_datetime('today')

driver.get("https://www.tefas.gov.tr/FonKarsilastirma.aspx")

time.sleep(10)

dropdown = driver.find_element(By.NAME, "table_fund_returns_length")
select = Select(dropdown)
select.select_by_value("250")

df_list = []
while True:
    # Sayfa kaynağını çek
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    table = soup.find('table', {'id': 'table_fund_returns'})
    html_content = str(table)
    df = pd.read_html(StringIO(html_content), thousands='.')[0]
    # Elde edilen veriyi listeye ekle
    df_list.append(df)
    
    # "Sonraki" butonunun durumunu kontrol et
    next_button = driver.find_element(By.ID, "table_fund_returns_next")
    if "disabled" in next_button.get_attribute("class"):
        break  # Eğer buton etkisizse döngüyü kır

    # "Sonraki" butonuna tıkla ve bekle
    next_button.click()
    time.sleep(1.3)

print("Sayfa / yeni sayfa çekimi tamamlandı...")
print("Birleştiriliyor...")
islemgoren = pd.concat(df_list, ignore_index=True)

driver.quit()

islemgoren

Sayfa / yeni sayfa çekimi tamamlandı...
Birleştiriliyor...


,Fon Kodu,Fon Adı,Şemsiye Fon Türü,1 Ay (%),3 Ay (%),6 Ay (%),Yılbaşı (%),1 Yıl (%),3 Yıl (%),5 Yıl (%)
0,DVT,DENİZ PORTFÖY METAVERSE VE DİJİTAL YAŞAM TEKNO...,Değişken Şemsiye Fonu,"8,9870","37,6033","51,4743","6,9775","185,3680",-,-
1,RTH,RE-PIE PORTFÖY BİRİNCİ HİSSE SENEDİ SERBEST FO...,Serbest Şemsiye Fonu,"13,6797","12,9803","40,1913","11,9471","183,8931",-,-
2,IIE,İSTANBUL PORTFÖY ONDÖRDÜNCÜ SERBEST FON,Serbest Şemsiye Fonu,"10,3063","5,9469","66,1849","7,4062","164,0389",-,-
3,IJZ,İŞ PORTFÖY SİBER GÜVENLİK TEKNOLOJİLERİ DEĞİŞK...,Değişken Şemsiye Fonu,"13,4651","35,2938","61,3451","11,4625","145,4438",-,-
4,IPB,İSTANBUL PORTFÖY BİRİNCİ DEĞİŞKEN FON,Değişken Şemsiye Fonu,"6,9743","14,4225","45,6456","3,5568","138,2925","803,1579","1.493,2431"
...,...,...,...,...,...,...,...,...,...,...
577,YLY,YAPI KREDİ PORTFÖY KİLYOS KAR PAYI ÖDEYEN HİSS...,Serbest Şemsiye Fonu,"11,4558","6,8021","29,7967","6,5552",-,-,-
578,YMD,YAPI KREDİ PORTFÖY MASLAK SERBEST (DÖVİZ- ABD ...,Serbest Şemsiye Fonu,"3,7328","9,7446","16,0158","2,8835",-,-,-
579,YNK,YAPI KREDİ PORTFÖY NİŞANTAŞI KAR PAYI ÖDEYEN S...,Serbest Şemsiye Fonu,"1,9609","12,0022","16,5222","1,0396",-,-,-
580,YTV,YAPI KREDİ PORTFÖY TARIM DEĞİŞKEN FON,Değişken Şemsiye Fonu,"1,1407","10,8161","12,1541","-1,2263",-,-,-


In [5]:
sureklicekimfonlar = pd.read_csv("sureklicekimfonlar.csv").reset_index(drop=True).iloc[:, 1:]

sureklicekim = pd.concat(
                [pd.read_csv("concated_mainyear_2020.csv").reset_index(drop=True).iloc[:, 1:],
               pd.read_csv("concated_mainyear_2021.csv").reset_index(drop=True).iloc[:, 1:],
               pd.read_csv("concated_mainyear_2022.csv").reset_index(drop=True).iloc[:, 1:],
               pd.read_csv("concated_mainyear_2023.csv").reset_index(drop=True).iloc[:, 1:],
               sureklicekimfonlar],
              ignore_index=True)
sureklicekim

,Tarih,Fon Kodu,Fon Adı,Fiyat,Tedavüldeki Pay Sayısı,Kişi Sayısı,Fon Toplam Değer
0,02.01.2020,AAK,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,35.677757,"456,001.0",303,"16,269,093.1"
1,02.01.2020,AAL,ATA PORTFÖY PARA PİYASASI (TL) FONU,0.575406,"201,742,677.0","3,046","116,083,937.71"
2,02.01.2020,AAS,ATA PORTFÖY FON SEPETİ SERBEST FONU,0.025357,"128,719,961.0",7,"3,263,938.64"
3,02.01.2020,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,0.031263,"2,868,807,200.0",11,"89,688,581.9"
4,02.01.2020,ABU,AK PORTFÖY BİREBİR BANKACILIK DEĞİŞKEN ÖZEL FON,0.019345,"612,106,546.0",117,"11,841,232.29"
...,...,...,...,...,...,...,...
883500,26.01.2024,ZVB,ATLAS PORTFÖY SERBEST (TL) FON,26.716488,"3,750,538.0",10,"100,201,202.01"
883501,26.01.2024,ZVO,ZİRAAT PORTFÖY ÜÇÜNCÜ SERBEST (TL) FON,2.648466,"235,410,837.0",12,"623,477,577.28"
883502,26.01.2024,ZYC,ZİRAAT PORTFÖY DÖRDÜNCÜ SERBEST (TL) FON,1.613509,"1,833,786,550.0",1,"2,958,831,978.27"
883503,26.01.2024,ZYD,ZİRAAT PORTFÖY BEŞİNCİ SERBEST (TL) FON,1.614778,"1,844,187,892.0",1,"2,977,953,504.35"


In [6]:
sureklicekim = sureklicekim.loc[sureklicekim["Fon Kodu"].isin(pd.DataFrame(islemgoren["Fon Kodu"].value_counts()).reset_index(drop=False).sort_values(by="Fon Kodu")["Fon Kodu"].unique())]

In [7]:
if pd.to_datetime('today').date().weekday() >= 5:  # Monday is 0, Sunday is 6
    print("haftasonu.")
            # Eksik tarihleri içeren yeni DataFrame oluştur
    start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
    end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
    sureklicekim["Tarih"] = pd.to_datetime(sureklicekim["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
    date_range = pd.date_range(start_date, end_date)
    df_all_dates = pd.DataFrame({'Tarih': date_range})


    
    sureklicekim_yapilandirilmis = pd.DataFrame()
    for fonkodu in pd.DataFrame(sureklicekim["Fon Kodu"].value_counts()).reset_index(drop=False).sort_values(by="Fon Kodu")["Fon Kodu"].unique():
        filtre = sureklicekim.loc[sureklicekim["Fon Kodu"] == fonkodu]
        filtre = filtre[filtre['Fiyat'] != "0.0"]
        filtre = filtre[filtre['Fiyat'] != "0,0"]
        filtre = filtre[filtre['Fiyat'] != "0"]
        filtre = filtre[filtre['Fiyat'] != 0]
        basim = pd.merge(df_all_dates, filtre, on="Tarih", how="left").infer_objects(copy=False).ffill().dropna()
        sureklicekim_yapilandirilmis = pd.concat([sureklicekim_yapilandirilmis, basim.sort_values(by="Tarih")])
    sureklicekim = sureklicekim_yapilandirilmis.sort_values(by=["Tarih","Fon Kodu"]).reset_index(drop=True)



    sureklicekim_yapilandirilmis = []
    df = []
    newdf = []
    sureklicekimfonlar = []
    print("data düzenlendi.")
    pass
else:
    if pd.to_datetime(sureklicekim['Tarih'], format="%d.%m.%Y").max().date() == pd.to_datetime('today').date():
        print("data mevcut.")
                # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        sureklicekim["Tarih"] = pd.to_datetime(sureklicekim["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})


        
        sureklicekim_yapilandirilmis = pd.DataFrame()
        for fonkodu in pd.DataFrame(sureklicekim["Fon Kodu"].value_counts()).reset_index(drop=False).sort_values(by="Fon Kodu")["Fon Kodu"].unique():
            filtre = sureklicekim.loc[sureklicekim["Fon Kodu"] == fonkodu]
            filtre = filtre[filtre['Fiyat'] != "0.0"]
            filtre = filtre[filtre['Fiyat'] != "0,0"]
            filtre = filtre[filtre['Fiyat'] != "0"]
            filtre = filtre[filtre['Fiyat'] != 0]
            basim = pd.merge(df_all_dates, filtre, on="Tarih", how="left").infer_objects(copy=False).ffill().dropna()
            sureklicekim_yapilandirilmis = pd.concat([sureklicekim_yapilandirilmis, basim.sort_values(by="Tarih")])
        sureklicekim = sureklicekim_yapilandirilmis.sort_values(by=["Tarih","Fon Kodu"]).reset_index(drop=True)



        sureklicekim_yapilandirilmis = []
        df = []
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")
        pass
    else:
        while True:
            driver = webdriver.Chrome()
        
            start_date = pd.to_datetime('today')
            end_date = pd.to_datetime('today')
        
            driver.get("https://www.tefas.gov.tr/TarihselVeriler.aspx")
            
            time.sleep(10)
            
            # Set start date
            driver.execute_script('document.getElementById("TextBoxStartDate").setAttribute("value", arguments[0])', start_date.strftime('%d.%m.%Y'))
        
            # Set end date
            driver.execute_script('document.getElementById("TextBoxEndDate").setAttribute("value", arguments[0])', end_date.strftime('%d.%m.%Y'))
        
            # Click the search button
            driver.find_element(By.ID, "ButtonSearchDates").click()
        
            print("Yükleniyor...")
        
            # Wait for the data to load (adjust the time.sleep value if needed)
            time.sleep(60)
        
            
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "td.dt-center.sorting_1")))
                print("Veriler girilmiş.")
        
                print("Sayfa yapılandırılıyor...")
        
                dropdown = driver.find_element(By.NAME, "table_general_info_length")
                select = Select(dropdown)
                select.select_by_value("250")
        
                print("Sayfa / yeni sayfa çekimine başlandı...")
                df_list = []
                while True:
                    # Sayfa kaynağını çek
                    page_source = driver.page_source
                    soup = BeautifulSoup(page_source, 'html.parser')
                    table = soup.find('table', {'id': 'table_general_info'})
                    html_content = str(table)
                    df = pd.read_html(StringIO(html_content), thousands='.')[0]
                    df["Tarih"] = pd.read_html(StringIO(html_content))[0]["Tarih"]
                    # Elde edilen veriyi listeye ekle
                    df_list.append(df)
        
                    # "Sonraki" butonunun durumunu kontrol et
                    next_button = driver.find_element(By.ID, "table_general_info_next")
                    if "disabled" in next_button.get_attribute("class"):
                        break  # Eğer buton etkisizse döngüyü kır
        
                    # "Sonraki" butonuna tıkla ve bekle
                    next_button.click()
                    time.sleep(1.3)
        
                print("Sayfa / yeni sayfa çekimi tamamlandı...")
                print("Birleştiriliyor...")
                df = pd.concat(df_list, ignore_index=True)
                
                print("Formatlanıyor...")
                newdf = df.copy()
                newdf['Tarih'] = pd.to_datetime(df['Tarih'], format="%d.%m.%Y")
                newdf['Tarih'] = newdf['Tarih'].dt.strftime("%d.%m.%Y")
                newdf['Fon Kodu'] = df['Fon Kodu'].astype(str)
                newdf['Fon Adı'] = df['Fon Adı'].astype(str)
                
                newdf['Fiyat'] = pd.to_numeric(df['Fiyat'].str.replace(".", "").str.replace(",", "."))
                newdf['Fiyat'] = newdf.apply(lambda x: "{:,}".format(x['Fiyat']), axis=1)
                
                newdf['Tedavüldeki Pay Sayısı'] = pd.to_numeric(df['Tedavüldeki Pay Sayısı'].str.replace(".", "").str.replace(",", "."))
                newdf['Tedavüldeki Pay Sayısı'] = newdf.apply(lambda x: "{:,}".format(x['Tedavüldeki Pay Sayısı']), axis=1)
                
                newdf['Fon Toplam Değer'] = pd.to_numeric(df['Fon Toplam Değer'].str.replace(".", "").str.replace(",", "."))
                newdf['Fon Toplam Değer'] = newdf.apply(lambda x: "{:,}".format(x['Fon Toplam Değer']), axis=1)
                
                newdf['Kişi Sayısı'] = pd.to_numeric(df['Kişi Sayısı'])
                newdf['Kişi Sayısı'] = newdf.apply(lambda x: "{:,}".format(x['Kişi Sayısı']), axis=1)
                
                
                print("Formatlama tamamlandı.")
                sureklicekimfonlar = pd.concat([sureklicekimfonlar,newdf],ignore_index=True)
                sureklicekimfonlar.to_csv("sureklicekimfonlar.csv")
                sureklicekim = pd.concat([sureklicekim,newdf],ignore_index=True)
    
                            # Eksik tarihleri içeren yeni DataFrame oluştur
                start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
                end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
                sureklicekim.loc["Tarih"] = pd.to_datetime(sureklicekim["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
                date_range = pd.date_range(start_date, end_date)
                df_all_dates = pd.DataFrame({'Tarih': date_range})


                
                sureklicekim_yapilandirilmis = pd.DataFrame()
                for fonkodu in pd.DataFrame(sureklicekim["Fon Kodu"].value_counts()).reset_index(drop=False).sort_values(by="Fon Kodu")["Fon Kodu"].unique():
                    filtre = sureklicekim.loc[sureklicekim["Fon Kodu"] == fonkodu]
                    filtre = filtre[filtre['Fiyat'] != "0.0"]
                    filtre = filtre[filtre['Fiyat'] != "0,0"]
                    filtre = filtre[filtre['Fiyat'] != "0"]
                    filtre = filtre[filtre['Fiyat'] != 0]
                    basim = pd.merge(df_all_dates, filtre, on="Tarih", how="left").infer_objects(copy=False).ffill().dropna()
                    sureklicekim_yapilandirilmis = pd.concat([sureklicekim_yapilandirilmis, basim.sort_values(by="Tarih")])
                sureklicekim = sureklicekim_yapilandirilmis.sort_values(by=["Tarih","Fon Kodu"]).reset_index(drop=True)



                sureklicekim_yapilandirilmis = []
                df = []
                newdf = []
                sureklicekimfonlar = []
                print("data düzenlendi.")
    
                
                break
                
        
            except:
                print("Veriler girilmemiş, 30 saniye sonra yeniden denenecek.")
                # Continue the loop for another attempt
                continue
        
            finally:
                
                time.sleep(30)
                driver.quit()

haftasonu.


C:\Users\emrecan.ulu.DESKTOP-4T87DPH\AppData\Local\Temp\ipykernel_16360\4266112317.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sureklicekim["Tarih"] = pd.to_datetime(sureklicekim["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur


data düzenlendi.


In [8]:
sureklicekim

,Tarih,Fon Kodu,Fon Adı,Fiyat,Tedavüldeki Pay Sayısı,Kişi Sayısı,Fon Toplam Değer
0,2020-01-02,AAK,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,35.677757,"456,001.0",303,"16,269,093.1"
1,2020-01-02,AAL,ATA PORTFÖY PARA PİYASASI (TL) FONU,0.575406,"201,742,677.0","3,046","116,083,937.71"
2,2020-01-02,AAS,ATA PORTFÖY FON SEPETİ SERBEST FONU,0.025357,"128,719,961.0",7,"3,263,938.64"
3,2020-01-02,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,0.031263,"2,868,807,200.0",11,"89,688,581.9"
4,2020-01-02,ACC,İSTANBUL PORTFÖY DÖRDÜNCÜ HİSSE SENEDİ FONU (H...,1.996601,"11,802,124.0",392,"23,564,131.3"
...,...,...,...,...,...,...,...
576523,2024-01-28,ZPE,ZİRAAT PORTFÖY KATILIM HİSSE SENEDİ FONU (HİSS...,17.63725,"33,458,136.84","6,082","590,109,512.59"
576524,2024-01-28,ZPF,ZİRAAT PORTFÖY KATILIM FONU (DÖVİZ),12.074488,"157,258,066.0","4,085","1,898,810,562.58"
576525,2024-01-28,ZPG,ZİRAAT PORTFÖY KİRA SERTİFİKALARI (SUKUK) KATI...,3.991076,"56,777,060.31","3,047","226,601,582.4"
576526,2024-01-28,ZSF,ZİRAAT PORTFÖY S&P/OIC COMCEC (İSEDAK) 50 SHAR...,30.825283,"661,899.0","1,621","20,403,224.15"


In [9]:
sureklicekim = sureklicekim.loc[sureklicekim["Fon Kodu"].isin(pd.DataFrame(islemgoren["Fon Kodu"].value_counts()).reset_index(drop=False).sort_values(by="Fon Kodu")["Fon Kodu"].unique())]
sureklicekim = pd.merge(sureklicekim, islemgoren[["Fon Kodu","Şemsiye Fon Türü"]], how="left", on="Fon Kodu")

In [10]:
sureklicekim

,Tarih,Fon Kodu,Fon Adı,Fiyat,Tedavüldeki Pay Sayısı,Kişi Sayısı,Fon Toplam Değer,Şemsiye Fon Türü
0,2020-01-02,AAK,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,35.677757,"456,001.0",303,"16,269,093.1",Değişken Şemsiye Fonu
1,2020-01-02,AAL,ATA PORTFÖY PARA PİYASASI (TL) FONU,0.575406,"201,742,677.0","3,046","116,083,937.71",Para Piyasası Şemsiye Fonu
2,2020-01-02,AAS,ATA PORTFÖY FON SEPETİ SERBEST FONU,0.025357,"128,719,961.0",7,"3,263,938.64",Serbest Şemsiye Fonu
3,2020-01-02,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,0.031263,"2,868,807,200.0",11,"89,688,581.9",Hisse Senedi Şemsiye Fonu
4,2020-01-02,ACC,İSTANBUL PORTFÖY DÖRDÜNCÜ HİSSE SENEDİ FONU (H...,1.996601,"11,802,124.0",392,"23,564,131.3",Hisse Senedi Şemsiye Fonu
...,...,...,...,...,...,...,...,...
576523,2024-01-28,ZPE,ZİRAAT PORTFÖY KATILIM HİSSE SENEDİ FONU (HİSS...,17.63725,"33,458,136.84","6,082","590,109,512.59",Hisse Senedi Şemsiye Fonu
576524,2024-01-28,ZPF,ZİRAAT PORTFÖY KATILIM FONU (DÖVİZ),12.074488,"157,258,066.0","4,085","1,898,810,562.58",Katılım Şemsiye Fonu
576525,2024-01-28,ZPG,ZİRAAT PORTFÖY KİRA SERTİFİKALARI (SUKUK) KATI...,3.991076,"56,777,060.31","3,047","226,601,582.4",Katılım Şemsiye Fonu
576526,2024-01-28,ZSF,ZİRAAT PORTFÖY S&P/OIC COMCEC (İSEDAK) 50 SHAR...,30.825283,"661,899.0","1,621","20,403,224.15",Hisse Senedi Şemsiye Fonu


# Altın Çekimi

In [11]:
altin = pd.read_csv("altin.csv").iloc[:,1:]
altin.columns = altin.columns.str.replace('_x', '')
altin

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,01.01.2020,"290,352","290,727","290,945","290,331",NaN,"0,01%"
1,02.01.2020,"292,842","290,356","293,185","290,185",NaN,"0,86%"
2,03.01.2020,"297,960","292,844","298,495","292,844",NaN,"1,75%"
3,05.01.2020,"301,218","300,737","304,721","300,737",NaN,"1,09%"
4,06.01.2020,"300,177","301,206","303,438","299,500",NaN,"-0,35%"
...,...,...,...,...,...,...,...
1269,22.01.2024,"1.966,523","1.969,516","1.973,866","1.962,361",NaN,-0.16%
1270,23.01.2024,"1.976,077","1.966,722","1.983,385","1.956,313",NaN,+0.49%
1271,24.01.2024,"1.961,354","1.975,881","1.979,937","1.953,375",NaN,-0.75%
1272,25.01.2024,"1.968,801","1.961,620","1.971,449","1.957,985",NaN,+0.38%


In [12]:
if pd.to_datetime('today').date().weekday() >= 5:  # Monday is 0, Sunday is 6
    print("haftasonu")
            # Eksik tarihleri içeren yeni DataFrame oluştur
    start_date = pd.to_datetime("01.01.2020", format="%d.%m.%Y")
    end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
    altin["Tarih"] = pd.to_datetime(altin["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
    date_range = pd.date_range(start_date, end_date)
    df_all_dates = pd.DataFrame({'Tarih': date_range})
    altin = pd.merge(df_all_dates, altin, on="Tarih", how="left")
    altin = altin.ffill()
    altin = altin.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
    df = []
    newdf = []
    sureklicekimfonlar = []
    print("data düzenlendi.")
    pass
else:
    if pd.to_datetime(altin['Tarih'], format="%d.%m.%Y").max().date() == pd.to_datetime('today').date():
        print("data mevcut")
                # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("01.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        altin["Tarih"] = pd.to_datetime(altin["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})
        altin = pd.merge(df_all_dates, altin, on="Tarih", how="left")
        altin = altin.ffill()
        altin = altin.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
        df = []
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")
        pass
    else:
        # Selenium WebDriver'ı başlat
        driver = webdriver.Chrome()  # Chrome tarayıcısı için
        url = "https://tr.investing.com/currencies/gau-try-historical-data"
        driver.get(url)
        
        # Tablonun olduğu HTML kodunu çek
        table_html = driver.find_element("css selector", ".freeze-column-w-1").get_attribute('outerHTML')
        
        # BeautifulSoup kullanarak HTML'i işle
        soup = BeautifulSoup(table_html, 'html.parser')
        
        # BeautifulSoup ile tabloyu dataframe'e çevir
        table = soup.find(name='table')
        html_string = str(table)
        
        # StringIO kullanarak HTML dizesini DataFrame'e çevir
        df = pd.read_html(StringIO(html_string))[0]
        
        # Selenium WebDriver'ı kapat
        driver.quit()
        altin = pd.merge(df, altin, on="Tarih",how="outer").reset_index(drop=True)
        altin.loc[altin["Şimdi_x"].isna(), "Şimdi_x"] = altin.loc[altin["Şimdi_x"].isna(), "Şimdi_y"]
        altin.loc[altin["Açılış_x"].isna(), "Açılış_x"] = altin.loc[altin["Açılış_x"].isna(), "Açılış_y"]
        altin.loc[altin["Yüksek_x"].isna(), "Yüksek_x"] = altin.loc[altin["Yüksek_x"].isna(), "Yüksek_y"]
        altin.loc[altin["Düşük_x"].isna(), "Düşük_x"] = altin.loc[altin["Düşük_x"].isna(), "Düşük_y"]
        altin.loc[altin["Hac._x"].isna(), "Hac._x"] = altin.loc[altin["Hac._x"].isna(), "Hac._y"]
        altin.loc[altin["Fark %_x"].isna(), "Fark %_x"] = altin.loc[altin["Fark %_x"].isna(), "Fark %_y"]
        altin["Tarih"] = pd.to_datetime(altin["Tarih"], format="%d.%m.%Y")
        altin = altin.sort_values(by="Tarih").reset_index(drop=True)
        altin["Tarih"] = altin["Tarih"].dt.strftime('%d.%m.%Y')
        altin = altin.iloc[:,:7]
        altin.to_csv("altin.csv")
            # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("01.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        altin["Tarih"] = pd.to_datetime(altin["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})
        altin = pd.merge(df_all_dates, altin, on="Tarih", how="left")
        altin = altin.ffill()
        altin = altin.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
        df = []
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")

haftasonu
data düzenlendi.


In [13]:
altin
altin.columns = altin.columns.str.replace('_x', '')
altin

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,2020-01-01,"290,352","290,727","290,945","290,331",NaN,"0,01%"
1,2020-01-02,"292,842","290,356","293,185","290,185",NaN,"0,86%"
2,2020-01-03,"297,960","292,844","298,495","292,844",NaN,"1,75%"
3,2020-01-04,"297,960","292,844","298,495","292,844",NaN,"1,75%"
4,2020-01-05,"301,218","300,737","304,721","300,737",NaN,"1,09%"
...,...,...,...,...,...,...,...
1484,2024-01-24,"1.961,354","1.975,881","1.979,937","1.953,375",NaN,-0.75%
1485,2024-01-25,"1.968,801","1.961,620","1.971,449","1.957,985",NaN,+0.38%
1486,2024-01-26,"1.965,424","1.968,836","1.974,488","1.963,420",NaN,-0.17%
1487,2024-01-27,"1.965,424","1.968,836","1.974,488","1.963,420",NaN,-0.17%


# Gümüş Çekimi

In [14]:
gumus = pd.read_csv("gumus.csv").iloc[:,1:]
gumus.columns = gumus.columns.str.replace('_x', '')
gumus

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,01.01.2020,34207.0,34102.0,34268.0,34180.0,NaN,"0,01%"
1,02.01.2020,34452.0,34464.0,34667.0,34139.0,NaN,"0,72%"
2,03.01.2020,34654.0,34548.0,35069.0,34511.0,NaN,"0,59%"
3,06.01.2020,34747.0,34826.0,35531.0,34615.0,NaN,"0,27%"
4,07.01.2020,35539.0,35256.0,35727.0,34454.0,NaN,"2,28%"
...,...,...,...,...,...,...,...
1058,22.01.2024,214451.0,214601.0,220051.0,213396.0,NaN,-2.34%
1059,23.01.2024,218262.0,218256.0,218729.0,214657.0,NaN,+1.78%
1060,24.01.2024,220111.0,219882.0,222913.0,217532.0,NaN,+0.85%
1061,25.01.2024,222949.0,223690.0,223974.0,220319.0,NaN,+1.29%


In [15]:
if pd.to_datetime('today').date().weekday() >= 5:  # Monday is 0, Sunday is 6
    print("ctesi")
            # Eksik tarihleri içeren yeni DataFrame oluştur
    start_date = pd.to_datetime("01.01.2020", format="%d.%m.%Y")
    end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
    gumus["Tarih"] = pd.to_datetime(gumus["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
    date_range = pd.date_range(start_date, end_date)
    df_all_dates = pd.DataFrame({'Tarih': date_range})
    gumus = pd.merge(df_all_dates, gumus, on="Tarih", how="left")
    gumus = gumus.ffill()
    gumus = gumus.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
    df = []
    newdf = []
    sureklicekimfonlar = []
    print("data düzenlendi.")
    pass
else:
    if pd.to_datetime(gumus['Tarih'], format="%d.%m.%Y").max().date() == pd.to_datetime('today').date():
        print("data mevcut")
                # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("01.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        gumus["Tarih"] = pd.to_datetime(gumus["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})
        gumus = pd.merge(df_all_dates, gumus, on="Tarih", how="left")
        gumus = gumus.ffill()
        gumus = gumus.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
        df = []
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")
        pass
    else:
        # Selenium WebDriver'ı başlat
        driver = webdriver.Chrome()  # Chrome tarayıcısı için
        url = "https://tr.investing.com/currencies/xagg-try-historical-data"
        driver.get(url)
        
        # Tablonun olduğu HTML kodunu çek
        table_html = driver.find_element("css selector", ".freeze-column-w-1").get_attribute('outerHTML')
        
        # BeautifulSoup kullanarak HTML'i işle
        soup = BeautifulSoup(table_html, 'html.parser')
        
        # BeautifulSoup ile tabloyu dataframe'e çevir
        table = soup.find(name='table')
        html_string = str(table)
        
        # StringIO kullanarak HTML dizesini DataFrame'e çevir
        df = pd.read_html(StringIO(html_string))[0]
        
        # Selenium WebDriver'ı kapat
        driver.quit()
        gumus = pd.merge(df, gumus, on="Tarih",how="outer").reset_index(drop=True)
        gumus.loc[gumus["Şimdi_x"].isna(), "Şimdi_x"] = gumus.loc[gumus["Şimdi_x"].isna(), "Şimdi_y"]
        gumus.loc[gumus["Açılış_x"].isna(), "Açılış_x"] = gumus.loc[gumus["Açılış_x"].isna(), "Açılış_y"]
        gumus.loc[gumus["Yüksek_x"].isna(), "Yüksek_x"] = gumus.loc[gumus["Yüksek_x"].isna(), "Yüksek_y"]
        gumus.loc[gumus["Düşük_x"].isna(), "Düşük_x"] = gumus.loc[gumus["Düşük_x"].isna(), "Düşük_y"]
        gumus.loc[gumus["Hac._x"].isna(), "Hac._x"] = gumus.loc[gumus["Hac._x"].isna(), "Hac._y"]
        gumus.loc[gumus["Fark %_x"].isna(), "Fark %_x"] = gumus.loc[gumus["Fark %_x"].isna(), "Fark %_y"]
        gumus["Tarih"] = pd.to_datetime(gumus["Tarih"], format="%d.%m.%Y")
        gumus = gumus.sort_values(by="Tarih").reset_index(drop=True)
        gumus["Tarih"] = gumus["Tarih"].dt.strftime('%d.%m.%Y')
        gumus = gumus.iloc[:,:7]
        gumus.to_csv("gumus.csv")
            # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("01.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        gumus["Tarih"] = pd.to_datetime(gumus["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})
        gumus = pd.merge(df_all_dates, gumus, on="Tarih", how="left")
        gumus = gumus.ffill()
        gumus = gumus.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
        df = []
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")

ctesi
data düzenlendi.


In [16]:
gumus
gumus.columns = gumus.columns.str.replace('_x', '')
gumus

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,2020-01-01,34207.0,34102.0,34268.0,34180.0,NaN,"0,01%"
1,2020-01-02,34452.0,34464.0,34667.0,34139.0,NaN,"0,72%"
2,2020-01-03,34654.0,34548.0,35069.0,34511.0,NaN,"0,59%"
3,2020-01-04,34654.0,34548.0,35069.0,34511.0,NaN,"0,59%"
4,2020-01-05,34654.0,34548.0,35069.0,34511.0,NaN,"0,59%"
...,...,...,...,...,...,...,...
1484,2024-01-24,220111.0,219882.0,222913.0,217532.0,NaN,+0.85%
1485,2024-01-25,222949.0,223690.0,223974.0,220319.0,NaN,+1.29%
1486,2024-01-26,221944.0,223249.0,225628.0,221326.0,NaN,-0.45%
1487,2024-01-27,221944.0,223249.0,225628.0,221326.0,NaN,-0.45%


# BIST100 ÇEKİMİ

In [17]:
bist100 = pd.read_csv("bist100.csv").iloc[:,1:]
bist100.columns = bist100.columns.str.replace('_x', '')
bist100

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,02.01.2020,"1.159,32","1.149,03","1.159,32","1.145,66","2,41B","1,32%"
1,03.01.2020,"1.136,84","1.151,43","1.155,71","1.126,17","2,93B","-1,94%"
2,06.01.2020,"1.114,13","1.125,32","1.129,73","1.112,12","2,04B","-2,00%"
3,07.01.2020,"1.126,00","1.125,94","1.133,87","1.123,48","1,78B","1,07%"
4,08.01.2020,"1.128,76","1.109,61","1.133,42","1.107,46","2,33B","0,25%"
...,...,...,...,...,...,...,...
1016,22.01.2024,"8.010,14","7.996,10","8.069,92","7.970,10","2,29B",+0.17%
1017,23.01.2024,"7.950,06","8.101,23","8.145,85","7.915,48","2,68B",-0.75%
1018,24.01.2024,"8.090,90","7.982,71","8.149,53","7.939,44","2,54B",+1.77%
1019,25.01.2024,"8.169,89","8.142,06","8.212,93","8.095,90","2,67B",+0.98%


In [18]:
if pd.to_datetime('today').date().weekday() >= 5:  # Monday is 0, Sunday is 6
    print("ctesi")
            # Eksik tarihleri içeren yeni DataFrame oluştur
    start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
    end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
    bist100["Tarih"] = pd.to_datetime(bist100["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
    date_range = pd.date_range(start_date, end_date)
    df_all_dates = pd.DataFrame({'Tarih': date_range})
    bist100 = pd.merge(df_all_dates, bist100, on="Tarih", how="left")
    bist100 = bist100.ffill()
    bist100 = bist100.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
    df = []
    newdf = []
    sureklicekimfonlar = []
    print("data düzenlendi.")
    pass
else:
    if pd.to_datetime(bist100['Tarih'], format="%d.%m.%Y").max().date() == pd.to_datetime('today').date():
        print("data mevcut")
                # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        bist100["Tarih"] = pd.to_datetime(bist100["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})
        bist100 = pd.merge(df_all_dates, bist100, on="Tarih", how="left")
        bist100 = bist100.ffill()
        bist100 = bist100.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
        df = []
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")
        pass
    else:
        # Selenium WebDriver'ı başlat
        driver = webdriver.Chrome()  # Chrome tarayıcısı için
        url = "https://tr.investing.com/indices/ise-100-historical-data"
        driver.get(url)
        
        # Tablonun olduğu HTML kodunu çek
        table_html = driver.find_element("css selector", ".freeze-column-w-1").get_attribute('outerHTML')
        
        # BeautifulSoup kullanarak HTML'i işle
        soup = BeautifulSoup(table_html, 'html.parser')
        
        # BeautifulSoup ile tabloyu dataframe'e çevir
        table = soup.find(name='table')
        html_string = str(table)
        
        # StringIO kullanarak HTML dizesini DataFrame'e çevir
        df = pd.read_html(StringIO(html_string))[0]
        
        # Selenium WebDriver'ı kapat
        driver.quit()
        bist100 = pd.merge(df, bist100, on="Tarih",how="outer").reset_index(drop=True)
        print(df)
        print(bist100)
        bist100.loc[bist100["Şimdi_x"].isna(), "Şimdi_x"] = bist100.loc[bist100["Şimdi_x"].isna(), "Şimdi_y"]
        bist100.loc[bist100["Açılış_x"].isna(), "Açılış_x"] = bist100.loc[bist100["Açılış_x"].isna(), "Açılış_y"]
        bist100.loc[bist100["Yüksek_x"].isna(), "Yüksek_x"] = bist100.loc[bist100["Yüksek_x"].isna(), "Yüksek_y"]
        bist100.loc[bist100["Düşük_x"].isna(), "Düşük_x"] = bist100.loc[bist100["Düşük_x"].isna(), "Düşük_y"]
        bist100.loc[bist100["Hac._x"].isna(), "Hac._x"] = bist100.loc[bist100["Hac._x"].isna(), "Hac._y"]
        bist100.loc[bist100["Fark %_x"].isna(), "Fark %_x"] = bist100.loc[bist100["Fark %_x"].isna(), "Fark %_y"]
        bist100["Tarih"] = pd.to_datetime(bist100["Tarih"], format="%d.%m.%Y")
        bist100 = bist100.sort_values(by="Tarih").reset_index(drop=True)
        bist100["Tarih"] = bist100["Tarih"].dt.strftime('%d.%m.%Y')
        bist100 = bist100.iloc[:,:7]
        bist100.to_csv("bist100.csv")
            # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        bist100["Tarih"] = pd.to_datetime(bist100["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})
        bist100 = pd.merge(df_all_dates, bist100, on="Tarih", how="left")
        bist100 = bist100.ffill()
        bist100 = bist100.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
        df = []
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")

ctesi
data düzenlendi.


In [19]:
bist100
bist100.columns = bist100.columns.str.replace('_x', '')
bist100

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,2020-01-02,"1.159,32","1.149,03","1.159,32","1.145,66","2,41B","1,32%"
1,2020-01-03,"1.136,84","1.151,43","1.155,71","1.126,17","2,93B","-1,94%"
2,2020-01-04,"1.136,84","1.151,43","1.155,71","1.126,17","2,93B","-1,94%"
3,2020-01-05,"1.136,84","1.151,43","1.155,71","1.126,17","2,93B","-1,94%"
4,2020-01-06,"1.114,13","1.125,32","1.129,73","1.112,12","2,04B","-2,00%"
...,...,...,...,...,...,...,...
1483,2024-01-24,"8.090,90","7.982,71","8.149,53","7.939,44","2,54B",+1.77%
1484,2024-01-25,"8.169,89","8.142,06","8.212,93","8.095,90","2,67B",+0.98%
1485,2024-01-26,"8.346,28","8.191,37","8.347,58","8.180,25","3,42B",+2.16%
1486,2024-01-27,"8.346,28","8.191,37","8.347,58","8.180,25","3,42B",+2.16%


# BIST30 ÇEKİMİ

In [20]:
bist30 = pd.read_csv("bist30.csv").iloc[:,1:]
bist30.columns = bist30.columns.str.replace('_x', '')
bist30

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,02.01.2020,"1.406,81","1.394,05","1.406,81","1.389,89","1,40B","1,33%"
1,03.01.2020,"1.380,25","1.396,87","1.403,15","1.370,28","1,63B","-1,89%"
2,06.01.2020,"1.355,14","1.367,39","1.373,27","1.352,44","1,08B","-1,82%"
3,07.01.2020,"1.365,75","1.369,60","1.376,24","1.362,46","907,81M","0,78%"
4,08.01.2020,"1.372,38","1.348,53","1.377,32","1.345,77","1,24B","0,49%"
...,...,...,...,...,...,...,...
1016,22.01.2024,"8.642,98","8.622,10","8.716,10","8.588,26","1,49B",+0.22%
1017,23.01.2024,"8.557,58","8.748,72","8.793,31","8.521,59","1,73B",-0.99%
1018,24.01.2024,"8.731,21","8.590,20","8.799,86","8.549,56","1,66B",+2.03%
1019,25.01.2024,"8.786,36","8.792,73","8.863,80","8.713,53","1,74B",+0.63%


In [21]:
if pd.to_datetime('today').date().weekday() >= 5:  # Monday is 0, Sunday is 6
    print("ctesi")
            # Eksik tarihleri içeren yeni DataFrame oluştur
    start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
    end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
    bist30["Tarih"] = pd.to_datetime(bist30["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
    date_range = pd.date_range(start_date, end_date)
    df_all_dates = pd.DataFrame({'Tarih': date_range})
    bist30 = pd.merge(df_all_dates, bist30, on="Tarih", how="left")
    bist30 = bist30.ffill()
    bist30 = bist30.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
    df = []
    newdf = []
    sureklicekimfonlar = []
    print("data düzenlendi.")
    pass
else:
    if pd.to_datetime(bist30['Tarih'], format="%d.%m.%Y").max().date() == pd.to_datetime('today').date():
        print("data mevcut")
                # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        bist30["Tarih"] = pd.to_datetime(bist30["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})
        bist30 = pd.merge(df_all_dates, bist30, on="Tarih", how="left")
        bist30 = bist30.ffill()
        bist30 = bist30.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
        df = []
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")
        pass
    else:
        # Selenium WebDriver'ı başlat
        driver = webdriver.Chrome()  # Chrome tarayıcısı için
        url = "https://tr.investing.com/indices/ise-30-historical-data"
        driver.get(url)
        
        # Tablonun olduğu HTML kodunu çek
        table_html = driver.find_element("css selector", ".freeze-column-w-1").get_attribute('outerHTML')
        
        # BeautifulSoup kullanarak HTML'i işle
        soup = BeautifulSoup(table_html, 'html.parser')
        
        # BeautifulSoup ile tabloyu dataframe'e çevir
        table = soup.find(name='table')
        html_string = str(table)
        
        # StringIO kullanarak HTML dizesini DataFrame'e çevir
        df = pd.read_html(StringIO(html_string))[0]
        
        # Selenium WebDriver'ı kapat
        driver.quit()
        bist30 = pd.merge(df, bist30, on="Tarih",how="outer").reset_index(drop=True)
        bist30.loc[bist30["Şimdi_x"].isna(), "Şimdi_x"] = bist30.loc[bist30["Şimdi_x"].isna(), "Şimdi_y"]
        bist30.loc[bist30["Açılış_x"].isna(), "Açılış_x"] = bist30.loc[bist30["Açılış_x"].isna(), "Açılış_y"]
        bist30.loc[bist30["Yüksek_x"].isna(), "Yüksek_x"] = bist30.loc[bist30["Yüksek_x"].isna(), "Yüksek_y"]
        bist30.loc[bist30["Düşük_x"].isna(), "Düşük_x"] = bist30.loc[bist30["Düşük_x"].isna(), "Düşük_y"]
        bist30.loc[bist30["Hac._x"].isna(), "Hac._x"] = bist30.loc[bist30["Hac._x"].isna(), "Hac._y"]
        bist30.loc[bist30["Fark %_x"].isna(), "Fark %_x"] = bist30.loc[bist30["Fark %_x"].isna(), "Fark %_y"]
        bist30["Tarih"] = pd.to_datetime(bist30["Tarih"], format="%d.%m.%Y")
        bist30 = bist30.sort_values(by="Tarih").reset_index(drop=True)
        bist30["Tarih"] = bist30["Tarih"].dt.strftime('%d.%m.%Y')
        bist30 = bist30.iloc[:,:7]
        bist30.to_csv("bist30.csv")
            # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        bist30["Tarih"] = pd.to_datetime(bist30["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})
        bist30 = pd.merge(df_all_dates, bist30, on="Tarih", how="left")
        bist30 = bist30.ffill()
        bist30 = bist30.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
        df = []
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")

ctesi
data düzenlendi.


In [22]:
bist30
bist30.columns = bist30.columns.str.replace('_x', '')
bist30

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,2020-01-02,"1.406,81","1.394,05","1.406,81","1.389,89","1,40B","1,33%"
1,2020-01-03,"1.380,25","1.396,87","1.403,15","1.370,28","1,63B","-1,89%"
2,2020-01-04,"1.380,25","1.396,87","1.403,15","1.370,28","1,63B","-1,89%"
3,2020-01-05,"1.380,25","1.396,87","1.403,15","1.370,28","1,63B","-1,89%"
4,2020-01-06,"1.355,14","1.367,39","1.373,27","1.352,44","1,08B","-1,82%"
...,...,...,...,...,...,...,...
1483,2024-01-24,"8.731,21","8.590,20","8.799,86","8.549,56","1,66B",+2.03%
1484,2024-01-25,"8.786,36","8.792,73","8.863,80","8.713,53","1,74B",+0.63%
1485,2024-01-26,"8.979,52","8.803,63","8.983,11","8.794,88","1,94B",+2.20%
1486,2024-01-27,"8.979,52","8.803,63","8.983,11","8.794,88","1,94B",+2.20%


# USD ÇEKİMİ

In [23]:
usd = pd.read_csv("usd.csv").iloc[:,1:]
usd.columns = usd.columns.str.replace('_x', '')
usd

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,01.01.2020,59466.0,59461.0,59491.0,59375.0,NaN,"0,06%"
1,02.01.2020,59566.0,59458.0,59615.0,59415.0,NaN,"0,04%"
2,03.01.2020,59723.0,59545.0,59785.0,59460.0,NaN,"0,02%"
3,05.01.2020,59674.0,59713.0,59768.0,59625.0,NaN,"-0,08%"
4,06.01.2020,59690.0,59758.0,59811.0,59631.0,NaN,"0,03%"
...,...,...,...,...,...,...,...
1151,22.01.2024,302656.0,302148.0,302745.0,301273.0,NaN,+0.19%
1152,23.01.2024,302843.0,302679.0,303087.0,299769.0,NaN,+0.06%
1153,24.01.2024,302530.0,302845.0,303589.0,300855.0,NaN,-0.10%
1154,25.01.2024,303020.0,302522.0,303975.0,302030.0,NaN,+0.16%


In [24]:
if pd.to_datetime('today').date().weekday() >= 5:  # Monday is 0, Sunday is 6
    print("haftasonu")
            # Eksik tarihleri içeren yeni DataFrame oluştur
    start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
    end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
    usd["Tarih"] = pd.to_datetime(usd["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
    date_range = pd.date_range(start_date, end_date)
    df_all_dates = pd.DataFrame({'Tarih': date_range})
    usd.drop_duplicates(subset='Tarih', keep='last', inplace=True)
    usd = pd.merge(df_all_dates, usd, on="Tarih", how="left")
    usd = usd.ffill()
    usd = usd.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
    df = []
    newdf = []
    sureklicekimfonlar = []
    print("data düzenlendi.")
    pass
else:
    if pd.to_datetime(usd['Tarih'], format="%d.%m.%Y").max().date() == pd.to_datetime('today').date():
        print("data mevcut")
                # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        usd["Tarih"] = pd.to_datetime(usd["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})
        usd.drop_duplicates(subset='Tarih', keep='last', inplace=True)
        usd = pd.merge(df_all_dates, usd, on="Tarih", how="left")
        usd = usd.ffill()
        usd = usd.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
        df = []
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")
        pass
    else:
        # Selenium WebDriver'ı başlat
        driver = webdriver.Chrome()  # Chrome tarayıcısı için
        url = "https://tr.investing.com/currencies/usd-try-historical-data"
        driver.get(url)
        
        # Tablonun olduğu HTML kodunu çek
        table_html = driver.find_element("css selector", ".freeze-column-w-1").get_attribute('outerHTML')
        
        # BeautifulSoup kullanarak HTML'i işle
        soup = BeautifulSoup(table_html, 'html.parser')
        
        # BeautifulSoup ile tabloyu dataframe'e çevir
        table = soup.find(name='table')
        html_string = str(table)
        
        # StringIO kullanarak HTML dizesini DataFrame'e çevir
        df = pd.read_html(StringIO(html_string))[0]
        df.drop_duplicates(subset='Tarih', keep='last', inplace=True)
        
        # Selenium WebDriver'ı kapat
        driver.quit()
        usd = pd.merge(df, usd, on="Tarih",how="outer").reset_index(drop=True)
        usd.loc[usd["Şimdi_x"].isna(), "Şimdi_x"] = usd.loc[usd["Şimdi_x"].isna(), "Şimdi_y"]
        usd.loc[usd["Açılış_x"].isna(), "Açılış_x"] = usd.loc[usd["Açılış_x"].isna(), "Açılış_y"]
        usd.loc[usd["Yüksek_x"].isna(), "Yüksek_x"] = usd.loc[usd["Yüksek_x"].isna(), "Yüksek_y"]
        usd.loc[usd["Düşük_x"].isna(), "Düşük_x"] = usd.loc[usd["Düşük_x"].isna(), "Düşük_y"]
        usd.loc[usd["Hac._x"].isna(), "Hac._x"] = usd.loc[usd["Hac._x"].isna(), "Hac._y"]
        usd.loc[usd["Fark %_x"].isna(), "Fark %_x"] = usd.loc[usd["Fark %_x"].isna(), "Fark %_y"]
        usd["Tarih"] = pd.to_datetime(usd["Tarih"], format="%d.%m.%Y")
        usd = usd.sort_values(by="Tarih").reset_index(drop=True)
        usd["Tarih"] = usd["Tarih"].dt.strftime('%d.%m.%Y')
        usd = usd.iloc[:,:7]
        usd.to_csv("usd.csv")
            # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        usd["Tarih"] = pd.to_datetime(usd["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})
        usd = pd.merge(df_all_dates, usd, on="Tarih", how="left")
        usd = usd.ffill()
        usd = usd.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")

haftasonu
data düzenlendi.


In [25]:
usd
usd.columns = usd.columns.str.replace('_x', '')


# FAİZ ÇEKİMİ

In [26]:
faizler = pd.read_csv("faizler.csv").iloc[:,1:]
faizler.columns = faizler.columns.str.replace('_x', '')
faizler

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,02.01.2020,1117.0,1117.0,1117.0,1117.0,NaN,"-1,67%"
1,03.01.2020,1110.0,1110.0,1110.0,1110.0,NaN,"-0,63%"
2,06.01.2020,1178.0,1178.0,1178.0,1178.0,NaN,"6,13%"
3,07.01.2020,1235.0,1235.0,1235.0,1235.0,NaN,"4,84%"
4,08.01.2020,1240.0,1240.0,1240.0,1240.0,NaN,"0,40%"
...,...,...,...,...,...,...,...
1020,22.01.2024,4370.0,4370.0,4370.0,4370.0,NaN,+0.02%
1021,23.01.2024,4347.0,4347.0,4347.0,4347.0,NaN,-0.53%
1022,24.01.2024,4315.0,4315.0,4315.0,4315.0,NaN,-0.73%
1023,25.01.2024,4203.0,4203.0,4203.0,4203.0,NaN,-2.61%


In [27]:
if pd.to_datetime('today').date().weekday() >= 5:  # Monday is 0, Sunday is 6
    print("haftasonu")
            # Eksik tarihleri içeren yeni DataFrame oluştur
    start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
    end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
    faizler["Tarih"] = pd.to_datetime(faizler["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
    date_range = pd.date_range(start_date, end_date)
    df_all_dates = pd.DataFrame({'Tarih': date_range})
    faizler.drop_duplicates(subset='Tarih', keep='last', inplace=True)
    faizler = pd.merge(df_all_dates, faizler, on="Tarih", how="left")
    faizler = faizler.ffill()
    faizler = faizler.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
    df = []
    newdf = []
    sureklicekimfonlar = []
    print("data düzenlendi.")
    pass
else:
    if pd.to_datetime(faizler['Tarih'], format="%d.%m.%Y").max().date() == pd.to_datetime('today').date():
        print("data mevcut")
                # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        faizler["Tarih"] = pd.to_datetime(faizler["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})
        faizler.drop_duplicates(subset='Tarih', keep='last', inplace=True)
        faizler = pd.merge(df_all_dates, faizler, on="Tarih", how="left")
        faizler = faizler.ffill()
        faizler = faizler.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
        df = []
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")
        pass
    else:
        # Selenium WebDriver'ı başlat
        driver = webdriver.Chrome()  # Chrome tarayıcısı için
        url = "https://tr.investing.com/indices/try-overnight-reference-rate-historical-data"
        driver.get(url)
        
        # Tablonun olduğu HTML kodunu çek
        table_html = driver.find_element("css selector", ".freeze-column-w-1").get_attribute('outerHTML')
        
        # BeautifulSoup kullanarak HTML'i işle
        soup = BeautifulSoup(table_html, 'html.parser')
        
        # BeautifulSoup ile tabloyu dataframe'e çevir
        table = soup.find(name='table')
        html_string = str(table)
        
        # StringIO kullanarak HTML dizesini DataFrame'e çevir
        df = pd.read_html(StringIO(html_string))[0]
        df.drop_duplicates(subset='Tarih', keep='last', inplace=True)
        
        # Selenium WebDriver'ı kapat
        driver.quit()
        faizler = pd.merge(df, faizler, on="Tarih",how="outer").reset_index(drop=True)
        faizler.loc[faizler["Şimdi_x"].isna(), "Şimdi_x"] = faizler.loc[faizler["Şimdi_x"].isna(), "Şimdi_y"]
        faizler.loc[faizler["Açılış_x"].isna(), "Açılış_x"] = faizler.loc[faizler["Açılış_x"].isna(), "Açılış_y"]
        faizler.loc[faizler["Yüksek_x"].isna(), "Yüksek_x"] = faizler.loc[faizler["Yüksek_x"].isna(), "Yüksek_y"]
        faizler.loc[faizler["Düşük_x"].isna(), "Düşük_x"] = faizler.loc[faizler["Düşük_x"].isna(), "Düşük_y"]
        faizler.loc[faizler["Hac._x"].isna(), "Hac._x"] = faizler.loc[faizler["Hac._x"].isna(), "Hac._y"]
        faizler.loc[faizler["Fark %_x"].isna(), "Fark %_x"] = faizler.loc[faizler["Fark %_x"].isna(), "Fark %_y"]
        faizler["Tarih"] = pd.to_datetime(faizler["Tarih"], format="%d.%m.%Y")
        faizler = faizler.sort_values(by="Tarih").reset_index(drop=True)
        faizler["Tarih"] = faizler["Tarih"].dt.strftime('%d.%m.%Y')
        faizler = faizler.iloc[:,:7]
        faizler.to_csv("faizler.csv")
            # Eksik tarihleri içeren yeni DataFrame oluştur
        start_date = pd.to_datetime("02.01.2020", format="%d.%m.%Y")
        end_date = pd.to_datetime(pd.to_datetime('today'), format="%d.%m.%Y")
        faizler["Tarih"] = pd.to_datetime(faizler["Tarih"], format="%d.%m.%Y")# Eksik tarihleri oluştur
        date_range = pd.date_range(start_date, end_date)
        df_all_dates = pd.DataFrame({'Tarih': date_range})
        faizler = pd.merge(df_all_dates, faizler, on="Tarih", how="left")
        faizler = faizler.ffill()
        faizler = faizler.reset_index(drop=True).sort_values(by=["Tarih"]).reset_index(drop=True)
        newdf = []
        sureklicekimfonlar = []
        print("data düzenlendi.")

haftasonu
data düzenlendi.


In [28]:
faizler

,Tarih,Şimdi,Açılış,Yüksek,Düşük,Hac.,Fark %
0,2020-01-02,1117.0,1117.0,1117.0,1117.0,NaN,"-1,67%"
1,2020-01-03,1110.0,1110.0,1110.0,1110.0,NaN,"-0,63%"
2,2020-01-04,1110.0,1110.0,1110.0,1110.0,NaN,"-0,63%"
3,2020-01-05,1110.0,1110.0,1110.0,1110.0,NaN,"-0,63%"
4,2020-01-06,1178.0,1178.0,1178.0,1178.0,NaN,"6,13%"
...,...,...,...,...,...,...,...
1483,2024-01-24,4315.0,4315.0,4315.0,4315.0,NaN,-0.73%
1484,2024-01-25,4203.0,4203.0,4203.0,4203.0,NaN,-2.61%
1485,2024-01-26,4350.0,4350.0,4350.0,4350.0,NaN,+3.52%
1486,2024-01-27,4350.0,4350.0,4350.0,4350.0,NaN,+3.52%


# FINAL DATA

In [29]:
df = sureklicekim.copy()
newdf = df.copy()

newdf['Fon Kodu'] = df['Fon Kodu'].astype(str)
newdf['Fon Adı'] = df['Fon Adı'].astype(str)

newdf['Fiyat'] = pd.to_numeric(df['Fiyat'].astype(str).str.replace(",", ""))
#newdf['Fiyat'] = newdf.apply(lambda x: "{:,}".format(x['Fiyat']), axis=1)

newdf['Tedavüldeki Pay Sayısı'] = pd.to_numeric(df['Tedavüldeki Pay Sayısı'].astype(str).str.replace(",", ""))
#newdf['Tedavüldeki Pay Sayısı'] = newdf.apply(lambda x: "{:,}".format(x['Tedavüldeki Pay Sayısı']), axis=1)

newdf['Fon Toplam Değer'] = pd.to_numeric(df['Fon Toplam Değer'].astype(str).str.replace(",", ""))
#newdf['Fon Toplam Değer'] = newdf.apply(lambda x: "{:,}".format(x['Fon Toplam Değer']), axis=1)

newdf['Kişi Sayısı'] = pd.to_numeric(df['Kişi Sayısı'].astype(str).str.replace(",", ""))
#newdf['Kişi Sayısı'] = newdf.apply(lambda x: "{:,}".format(x['Kişi Sayısı']), axis=1)

newdf = newdf.sort_values(by=['Fon Kodu', 'Tarih'])
newdf['Günlük Getiri'] = newdf.groupby('Fon Kodu')['Fiyat'].pct_change()
newdf['Günlük Para Hareketi'] = newdf['Fon Toplam Değer'] - ((1 + newdf['Günlük Getiri']) * newdf.groupby('Fon Kodu')['Fon Toplam Değer'].shift(1))
newdf = newdf.sort_values(by=["Tarih","Fon Kodu"]).reset_index(drop=True)


In [30]:
usd["Yüksek"] = usd["Yüksek"].astype(str).str.replace(".", "")
usd["Yüksek"] = usd["Yüksek"].str[:-5] + "." + usd["Yüksek"].str[-5:]
usd["Yüksek"] = pd.to_numeric(usd["Yüksek"])
newdf = pd.merge(newdf, usd[["Tarih","Yüksek"]], on='Tarih', how='left').rename(columns={"Yüksek":"ForexBuyingUSD"})
newdf['FiyatUSD'] = newdf['Fiyat'] / newdf['ForexBuyingUSD']

In [31]:
altin["Yüksek"] = altin["Yüksek"].astype(str).str.replace(".", "")
altin["Yüksek"] = altin["Yüksek"].astype(str).str.replace(",", ".")
altin["Yüksek"] = pd.to_numeric(altin["Yüksek"])
newdf = pd.merge(newdf, altin[["Tarih","Yüksek"]], on='Tarih', how='left').rename(columns={"Yüksek":"Gram Altın Fiyatı TL"})

In [32]:
gumus["Yüksek"] = gumus["Yüksek"].astype(str).str.replace(".", "")
gumus["Yüksek"] = gumus["Yüksek"].str[:-5] + "." + gumus["Yüksek"].str[-5:]
gumus["Yüksek"] = pd.to_numeric(gumus["Yüksek"])
newdf = pd.merge(newdf, gumus[["Tarih","Yüksek"]], on='Tarih', how='left').rename(columns={"Yüksek":"Gram Gümüş Fiyatı TL"})

In [33]:
bist100["Yüksek"] = bist100["Yüksek"].astype(str).str.replace(".", "")
bist100["Yüksek"] = bist100["Yüksek"].astype(str).str.replace(",", ".")
bist100["Yüksek"] = pd.to_numeric(bist100["Yüksek"])
newdf = pd.merge(newdf, bist100[["Tarih","Yüksek"]], on='Tarih', how='left').rename(columns={"Yüksek":"Bist 100 Fiyatı"})

In [34]:
bist30["Yüksek"] = bist30["Yüksek"].astype(str).str.replace(".", "")
bist30["Yüksek"] = bist30["Yüksek"].astype(str).str.replace(",", ".")
bist30["Yüksek"] = pd.to_numeric(bist30["Yüksek"])
newdf = pd.merge(newdf, bist30[["Tarih","Yüksek"]], on='Tarih', how='left').rename(columns={"Yüksek":"Bist 30 Fiyatı"})

In [35]:
usd = usd[["Tarih","Yüksek"]].rename(columns={"Yüksek": "Fiyat"})
usd["Fon Kodu"] = "USDX"
usd["Fon Adı"] = "USD'nin TL Karşılığı"

altin = altin[["Tarih","Yüksek"]].rename(columns={"Yüksek": "Fiyat"})
altin["Fon Kodu"] = "ALTINX"
altin["Fon Adı"] = "Gram Altın Fiyatı TL"

gumus = gumus[["Tarih","Yüksek"]].rename(columns={"Yüksek": "Fiyat"})
gumus["Fon Kodu"] = "GUMUSX"
gumus["Fon Adı"] = "Gram Gümüş Fiyatı TL"

bist100 = bist100[["Tarih","Yüksek"]].rename(columns={"Yüksek": "Fiyat"})
bist100["Fon Kodu"] = "BIST100X"
bist100["Fon Adı"] = "BIST100 Fiyatı TL"

bist30 = bist30[["Tarih","Yüksek"]].rename(columns={"Yüksek": "Fiyat"})
bist30["Fon Kodu"] = "BIST30X"
bist30["Fon Adı"] = "BIST30 Fiyatı TL"


newdf = pd.concat([newdf,usd,altin,gumus,bist100,bist30],axis=0)



In [36]:
newdf

,Tarih,Fon Kodu,Fon Adı,Fiyat,Tedavüldeki Pay Sayısı,Kişi Sayısı,Fon Toplam Değer,Şemsiye Fon Türü,Günlük Getiri,Günlük Para Hareketi,ForexBuyingUSD,FiyatUSD,Gram Altın Fiyatı TL,Gram Gümüş Fiyatı TL,Bist 100 Fiyatı,Bist 30 Fiyatı
0,2020-01-02,AAK,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,35.677757,4.560010e+05,303.0,1.626909e+07,Değişken Şemsiye Fonu,NaN,NaN,5.9615,5.984695,293.185,3.4667,1159.32,1406.81
1,2020-01-02,AAL,ATA PORTFÖY PARA PİYASASI (TL) FONU,0.575406,2.017427e+08,3046.0,1.160839e+08,Para Piyasası Şemsiye Fonu,NaN,NaN,5.9615,0.096520,293.185,3.4667,1159.32,1406.81
2,2020-01-02,AAS,ATA PORTFÖY FON SEPETİ SERBEST FONU,0.025357,1.287200e+08,7.0,3.263939e+06,Serbest Şemsiye Fonu,NaN,NaN,5.9615,0.004253,293.185,3.4667,1159.32,1406.81
3,2020-01-02,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,0.031263,2.868807e+09,11.0,8.968858e+07,Hisse Senedi Şemsiye Fonu,NaN,NaN,5.9615,0.005244,293.185,3.4667,1159.32,1406.81
4,2020-01-02,ACC,İSTANBUL PORTFÖY DÖRDÜNCÜ HİSSE SENEDİ FONU (H...,1.996601,1.180212e+07,392.0,2.356413e+07,Hisse Senedi Şemsiye Fonu,NaN,NaN,5.9615,0.334916,293.185,3.4667,1159.32,1406.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,2024-01-24,BIST30X,BIST30 Fiyatı TL,8799.860000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1484,2024-01-25,BIST30X,BIST30 Fiyatı TL,8863.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1485,2024-01-26,BIST30X,BIST30 Fiyatı TL,8983.110000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1486,2024-01-27,BIST30X,BIST30 Fiyatı TL,8983.110000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
newdf.to_csv("newdf3.csv", index=False, sep='\t', encoding="utf-8", decimal=',')

In [38]:
newdf.loc[newdf["Fon Kodu"] == "IPB"]

,Tarih,Fon Kodu,Fon Adı,Fiyat,Tedavüldeki Pay Sayısı,Kişi Sayısı,Fon Toplam Değer,Şemsiye Fon Türü,Günlük Getiri,Günlük Para Hareketi,ForexBuyingUSD,FiyatUSD,Gram Altın Fiyatı TL,Gram Gümüş Fiyatı TL,Bist 100 Fiyatı,Bist 30 Fiyatı
125,2020-01-02,IPB,İSTANBUL PORTFÖY BİRİNCİ DEĞİŞKEN FON,0.028836,2.708144e+07,65.0,7.809276e+05,Değişken Şemsiye Fonu,NaN,NaN,5.9615,0.004837,293.185,3.4667,1159.32,1406.81
364,2020-01-03,IPB,İSTANBUL PORTFÖY BİRİNCİ DEĞİŞKEN FON,0.028844,2.645949e+07,68.0,7.631922e+05,Değişken Şemsiye Fonu,0.000277,-1.795205e+04,5.9785,0.004825,298.495,3.5069,1155.71,1403.15
604,2020-01-04,IPB,İSTANBUL PORTFÖY BİRİNCİ DEĞİŞKEN FON,0.028844,2.645949e+07,68.0,7.631922e+05,Değişken Şemsiye Fonu,0.000000,0.000000e+00,5.9785,0.004825,298.495,3.5069,1155.71,1403.15
844,2020-01-05,IPB,İSTANBUL PORTFÖY BİRİNCİ DEĞİŞKEN FON,0.028844,2.645949e+07,68.0,7.631922e+05,Değişken Şemsiye Fonu,0.000000,0.000000e+00,5.9768,0.004826,304.721,3.5069,1155.71,1403.15
1084,2020-01-06,IPB,İSTANBUL PORTFÖY BİRİNCİ DEĞİŞKEN FON,0.028867,2.732794e+07,63.0,7.888627e+05,Değişken Şemsiye Fonu,0.000797,2.506191e+04,5.9811,0.004826,303.438,3.5531,1129.73,1373.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573905,2024-01-24,IPB,İSTANBUL PORTFÖY BİRİNCİ DEĞİŞKEN FON,0.382819,3.939511e+09,28546.0,1.508120e+09,Değişken Şemsiye Fonu,0.001800,-1.145810e+07,30.3589,0.012610,1979.937,22.2913,8149.53,8799.86
574484,2024-01-25,IPB,İSTANBUL PORTFÖY BİRİNCİ DEĞİŞKEN FON,0.386124,3.927016e+09,28478.0,1.516314e+09,Değişken Şemsiye Fonu,0.008633,-4.826657e+06,30.3975,0.012702,1971.449,22.3974,8212.93,8863.80
575064,2024-01-26,IPB,İSTANBUL PORTFÖY BİRİNCİ DEĞİŞKEN FON,0.389532,3.909267e+09,28458.0,1.522786e+09,Değişken Şemsiye Fonu,0.008826,-6.910415e+06,30.6496,0.012709,1974.488,22.5628,8347.58,8983.11
575644,2024-01-27,IPB,İSTANBUL PORTFÖY BİRİNCİ DEĞİŞKEN FON,0.389532,3.909267e+09,28458.0,1.522786e+09,Değişken Şemsiye Fonu,0.000000,0.000000e+00,30.6496,0.012709,1974.488,22.5628,8347.58,8983.11


In [39]:
gumus

,Tarih,Fiyat,Fon Kodu,Fon Adı
0,2020-01-01,3.4268,GUMUSX,Gram Gümüş Fiyatı TL
1,2020-01-02,3.4667,GUMUSX,Gram Gümüş Fiyatı TL
2,2020-01-03,3.5069,GUMUSX,Gram Gümüş Fiyatı TL
3,2020-01-04,3.5069,GUMUSX,Gram Gümüş Fiyatı TL
4,2020-01-05,3.5069,GUMUSX,Gram Gümüş Fiyatı TL
...,...,...,...,...
1484,2024-01-24,22.2913,GUMUSX,Gram Gümüş Fiyatı TL
1485,2024-01-25,22.3974,GUMUSX,Gram Gümüş Fiyatı TL
1486,2024-01-26,22.5628,GUMUSX,Gram Gümüş Fiyatı TL
1487,2024-01-27,22.5628,GUMUSX,Gram Gümüş Fiyatı TL
